In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else device

# Hyperparameters
batch_size = 32
block_size = 8
max_iters = 10000
eval_interval = 300
learning_rate = 1e-3
eval_iters = 200

In [2]:
# Generate Vocabulary
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))

print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
# Character-Level Tokenizer
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [ string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [4]:
# Train-Test Split
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

# Input-Target Parallel Implementation
def get_batch(split):
    data = train_data if split == "train" else val_data
    idx = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    x, y = x.to(device), y.to(device)
    
    return x, y

x, y = get_batch('train')
print("inputs: ")
print(x)
print("targets: ")
print(y)

inputs: 
tensor([[ 1, 39,  1, 61, 53, 56, 42,  1],
        [10,  1, 63, 53, 59,  1, 51, 39],
        [50,  1, 58, 43, 50, 50,  1, 63],
        [ 1, 47, 58,  1, 53, 59, 58,  1],
        [25, 17, 30, 15, 33, 32, 21, 27],
        [ 1, 47, 58,  1, 58, 53,  1, 51],
        [60, 43, 52,  1, 52, 53, 61,  6],
        [56,  1, 58, 56, 43, 57, 54, 39],
        [57, 46, 43,  1, 40, 56, 47, 52],
        [21,  1, 61, 39, 57, 11,  1, 61],
        [51,  6,  1, 57, 47, 56,  8,  0],
        [ 1, 42, 53, 53, 51, 57,  7, 42],
        [56, 39, 52, 45, 43, 50, 63,  1],
        [46, 43,  1, 41, 39, 52, 53, 54],
        [39, 51, 12,  0,  0, 16, 33, 15],
        [ 1, 46, 39, 57, 58,  1, 42, 53],
        [34, 21, 26, 15, 17, 26, 32, 21],
        [52, 45, 43,  1, 50, 39, 61, 10],
        [ 0, 21,  1, 46, 39, 60, 43,  1],
        [44, 53, 53, 58,  6,  0, 13, 52],
        [53, 58,  1, 41, 46, 53, 53, 57],
        [42, 43, 56,  1, 42, 53, 45,  2],
        [ 0, 32, 46, 43,  1, 44, 39, 56],
        [59, 57, 58,  1, 

In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(idx)
            # take last prediction 
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f'step: {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}')
    
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step: 0: train loss 2.4566, val loss 2.4991
step: 300: train loss 2.4593, val loss 2.5097
step: 600: train loss 2.4505, val loss 2.5124
step: 900: train loss 2.4549, val loss 2.5087
step: 1200: train loss 2.4551, val loss 2.5085
step: 1500: train loss 2.4432, val loss 2.5032
step: 1800: train loss 2.4540, val loss 2.5088
step: 2100: train loss 2.4494, val loss 2.5093
step: 2400: train loss 2.4530, val loss 2.5044
step: 2700: train loss 2.4533, val loss 2.5159
step: 3000: train loss 2.4535, val loss 2.5020
step: 3300: train loss 2.4441, val loss 2.5129
step: 3600: train loss 2.4580, val loss 2.5052
step: 3900: train loss 2.4480, val loss 2.5186
step: 4200: train loss 2.4554, val loss 2.5089
step: 4500: train loss 2.4527, val loss 2.5135
step: 4800: train loss 2.4572, val loss 2.5036
step: 5100: train loss 2.4423, val loss 2.5115
step: 5400: train loss 2.4471, val loss 2.5071
step: 5700: train loss 2.4494, val loss 2.5070
step: 6000: train loss 2.4465, val loss 2.5085
step: 6300: train l

In [23]:
# Generate the new tokens
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


An'de acono I'd shthor tinowerom,
F IN: othy mbe ans: m!


HAy'lderthithe he aithiny,

CUSofelde ppait wlors in:
Myond ges olieneve tha-nkedugefokecor LIIClo tows, grse pangobond e izl Hero Goughile eray otho wnourit che nd pe or:
NEThofothere ws no m.
S:
OMe umbin chis the s it'wad;
Wighanyofar f ne, t? ttin t s; r!
Lour ainghr s tirsorstepe thaulo hen h lan.
Bis beary omoundou, he:

CEST:
NESiceaveerege fott wisuo. burts PAl ban dedlel frer ordeeastert waghene therd.

MAs' bes ttteay se
Is wne
